In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/research'

In [2]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    results: Path
    experiment_name: str
    model_name: str
    dataset_yaml: Path
    model_params: dict

In [4]:
from src.cellseg.constant import *
from src.cellseg.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model
        
        create_directories([config.results])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            results=config.results,
            experiment_name=config.experiment_name,
            model_name=config.model_name,
            dataset_yaml=config.dataset_yaml,
            model_params=params
        )
        
        return model_trainer_config

In [5]:
from src.cellseg import logger
import yaml
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        model = YOLO(os.path.join(self.config.root_dir, self.config.model_name))
        
        results = model.train(
            data=self.config.dataset_yaml,
            project=self.config.results,
            name=self.config.experiment_name,
            epochs=self.config.model_params.epochs,
            patience=self.config.model_params.patience,
            batch=self.config.model_params.batch,
            imgsz=self.config.model_params.imgsz,
            device=self.config.model_params.device,
            workers=self.config.model_params.workers,
            pretrained=self.config.model_params.pretrained,
            optimizer=self.config.model_params.optimizer,
            verbose=self.config.model_params.verbose,
            deterministic=self.config.model_params.deterministic,
            cos_lr=self.config.model_params.cos_lr,
            close_mosaic=self.config.model_params.close_mosaic,
            freeze=self.config.model_params.freeze,
            lr0=self.config.model_params.lr0,
            lrf=self.config.model_params.lrf,
            momentum=self.config.model_params.momentum,
            weight_decay=self.config.model_params.weight_decay,
            warmup_epochs=self.config.model_params.warmup_epochs,
            warmup_bias_lr=self.config.model_params.warmup_bias_lr,
            dropout=self.config.model_params.dropout,
            plots=self.config.model_params.plots
        )

In [6]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

[2024-11-12 21:59:12,648: INFO: main_utils: created directory at: artifacts]
[2024-11-12 21:59:12,649: INFO: main_utils: created directory at: artifacts/model_trainer/results]
Ultralytics 8.3.29 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6065MiB)
engine/trainer: task=segment, mode=train, model=artifacts/model_trainer/yolov8n-seg.pt, data=artifacts/data_transformation/dataset.yaml, epochs=200, time=None, patience=0, batch=32, imgsz=256, save=True, save_period=-1, cache=False, device=0, workers=4, project=artifacts/model_trainer/results, name=200_epochs_5, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, so

[W1112 21:59:13.161666359 NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


YOLOv8n-seg summary: 261 layers, 3,263,811 parameters, 3,263,795 gradients, 12.1 GFLOPs

Transferred 381/417 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Cell_Segmentation_YOLO-v8/artifacts/data_transformation/train.cache... 2680 images, 4 backgrounds, 0 corrupt: 100%|██████████| 2680/2680 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

Plotting labels to artifacts/model_trainer/results/200_epochs_5/labels.jpg... 
optimizer: Adam(lr=0.01, momentum=0.937) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 4 dataloader workers
Logging results to artifacts/model_trainer/results/200_epochs_5
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/200      1.45G      3.842      4.719      2.585      1.613       1054        256: 100%|██████████| 84/84 [01:34<00:00,  1.13s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:11<00:00,  1.01s/it]


                   all        670      23245    0.00474      0.041    0.00281   0.000672    0.00565     0.0489    0.00299   0.000761

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      2/200      1.42G       3.66      4.378      2.402      1.432       1464        256:  17%|█▋        | 14/84 [00:14<01:10,  1.01s/it]


KeyboardInterrupt: 